In [4]:
from casatools import componentlist, image, coordsys, quanta
from casatasks import exportfits, simalma, tclean, imstat, imfit

cl = componentlist()
ia = image()
qa = quanta()

## Skymodel

In [7]:
PROJECT_NAME = "BAND-9"

dir = "ICRS 13h11m29.96s -01d19m18.7s"

cl.done()
cl.addcomponent(
    dir=dir, 
    flux=1.43e-3, 
    fluxunit='Jy', 
    freq='694GHz', 
    shape='Gaussian',
    majoraxis="0.81arcsec", 
    minoraxis="0.38arcsec", 
    positionangle="44.0deg"
)

ia.fromshape(f'{PROJECT_NAME}-SKYMODEL.im', [512,512,1,1], overwrite=True)

cs = ia.coordsys()
cs.setunits(['rad', 'rad', '', 'Hz'])

cell_rad = qa.convert(qa.quantity("0.01arcsec"), "rad")['value']
cs.setincrement([-cell_rad, cell_rad], 'direction') 

cs.setreferencevalue([qa.convert("13h11m29.96s", "rad")['value'],
qa.convert("-01d19m18.7s", "rad")['value']], type="direction")
cs.setreferencevalue("694GHz", "spectral") 
cs.setincrement("7.2GHz", "spectral")

ia.setcoordsys(cs.torecord())

ia.setbrightnessunit("Jy/pixel")

ia.modify(cl.torecord(), subtract=False)

ia.done()

exportfits(imagename=f"{PROJECT_NAME}-SKYMODEL.im", fitsimage=f"{PROJECT_NAME}-SKYMODEL.fits", overwrite=True)

## simalma

In [ ]:
simalma(
    project=PROJECT_NAME,
    skymodel=f'{PROJECT_NAME}-SKYMODEL.im',
    incenter='694GHz',
    inwidth='7.2GHz', 
    hourangle='transit',
    integration='10s',
    totaltime='5700s',
    antennalist=['alma.cycle6.2.cfg'],
    tpnant=0,
    pwv=0.85,
    cell='0.01arcsec',
    indirection=dir,
    imsize=[512,512],
    image=True,
    setpointings=True,
    graphics='both',
    verbose=True,
    overwrite=True,
    dryrun=False
)

## tclean

In [ ]:
tclean(
    vis=f'{PROJECT_NAME}/{PROJECT_NAME}.alma.cycle6.2.noisy.ms',
    imagename='CLEAN',
    weighting='natural',
    cell='0.01arcsec', 
    imsize=[512,512],
    deconvolver='multiscale',
    niter=25000,
    phasecenter=dir,
    threshold='210uJy',
    uvtaper=['0.5arcsec'],
    interactive=False
)

## imview

1. Run `imview()` in the CASA CLI, which opens a GUI. 
2. Summon a raster image and contour map on `CLEAN.image`!
3. Open up `Adjust Data Display`.
4. Navigate to the `CLEAN.image` tab and adjust `Relative Contour Levels`.
5. Set `Unit Contour Level` to your `RMS`.
6. Draw a region around the susepcted source.
7. Navigate to the `File` tab and save your region as `SOURCE.crtf`.
8. Repeat these actions for the `PACMAN.crtf`.

Use the helper below to send results to a `DATA.json`

In [ ]:
import json,numpy as np
f=lambda d:json.dump((g:=lambda x:x.tolist()if isinstance(x,np.ndarray)else x.item()if isinstance(x,np.generic)else{x:g(v)for x,v in x.items()}if isinstance(x,dict)else[list(map(g,x))]if isinstance(x,(list,tuple))else x)(d),open("DATA.json","w"),indent=4)

## imstat

In [ ]:
f(imstat(imagename='CLEAN.image', region='PACMAN.crtf'))

## imfit

In [ ]:
f(imfit(imagename='CLEAN.image', region='SOURCE.crtf'))